In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import os
from bs4 import BeautifulSoup
import re

import nltk.corpus
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')

from textblob import TextBlob
import html2text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
os.chdir(r"C:\Users\swani\Python Projects\Case Bucket\Data")

In [3]:
data = pd.read_excel('KBArticleReport.xlsx')
length = int(len(data))
length

3340

In [4]:
data

,Name,Answer__c,CreatedDate,Article_Number__c,Author_Region__c,Cadence_Platform__c,Channel__c,Critical__c,Definition__c,External_Family__c,...,Problem__c,Product_Feature2__r.Name,Product_Feature_Lvl_1__r.Name,Product_Feature_Lvl_2__r.Name,Product_Feature_Version__r.Name,Product_Feature__r.Name,Product__r.Name,Published_Date__c,Saved_Solution__c,Solution__c
0,20467935,NaN,2017-06-13T06:27:28.000+0000,538672.0,GCS,Functional Verification,Customer,0.0,<p><strong>For VHDL:</strong></p>\n\n<p> Yo...,Xcelium,...,NaN,Single Core,Multi-Language Sim,LRM Compliance,None,Single Core,Xcelium,2017-07-14T09:00:55.000+0000,<p>This is correct behavior as per the LRM as ...,<p>This is correct behavior as per the LRM as ...
1,20467953,NaN,2017-06-13T21:36:58.000+0000,538690.0,NACS,Functional Verification,Customer,0.0,NaN,Xcelium,...,<p>User is attempting to set an enumerated typ...,Single Core,UVM,UVM SV,None,Single Core,Xcelium,2017-06-13T22:11:03.000+0000,<p>It is not possible to directly pass an enum...,<p>It is not possible to directly pass an enum...
2,20467959,NaN,2017-06-14T13:39:23.000+0000,538696.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,NaN,Multi Core,Documentation,NaN,20.03,Multi Core,Xcelium,2017-06-14T13:39:23.000+0000,NaN,NaN
3,20467982,NaN,2017-06-15T11:18:38.000+0000,538719.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,NaN,Single Core,Comprehensive Coverage,Simulation/Recording,20.03,Single Core,Xcelium,2017-06-15T11:18:38.000+0000,NaN,NaN
4,20467983,NaN,2017-06-15T11:49:36.000+0000,538720.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,NaN,Functional Safety,Documentation,NaN,17.04,Functional Safety,Xcelium,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,20464289,"<p>Most likely, the issue is in the user C cod...",2017-02-15T06:39:10.000+0000,535026.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,NaN,Enterprise Simulator,SystemVerilog Sim,Simulator Engine,None,Single Core,Xcelium,2017-04-04T13:30:34.000+0000,NaN,NaN
3336,20464503,NaN,2017-02-21T05:27:25.000+0000,535240.0,NACS,Functional Verification,Customer,0.0,NaN,Xcelium,...,<p>The coverage database that was dumped from ...,Enterprise Simulator,Comprehensive Coverage,Functional,None,Single Core,Xcelium,2017-03-01T13:28:57.000+0000,<p>Assertions in simulation can be turned off ...,<p>Assertions in simulation can be turned off ...
3337,20464541,NaN,2017-02-22T09:21:36.000+0000,535278.0,GCS,Functional Verification,Customer,0.0,"<p>The &quot;<span style=""font-family: courier...",Xcelium,...,NaN,Enterprise Simulator,UVM,UVM_REG,None,Single Core,Xcelium,2017-04-13T07:51:44.000+0000,<p>If you want to enable coverage for the envi...,<p>If you want to enable coverage for the envi...
3338,20464805,<p>There is an option called &quot;<span style...,2017-02-25T11:54:23.000+0000,535542.0,NACS,Functional Verification,Customer,0.0,NaN,Xcelium,...,NaN,Enterprise Simulator,Multi-Language Sim,TCL,None,Single Core,Xcelium,2017-03-07T20:14:28.000+0000,NaN,NaN


In [33]:
# data['Solution__c'] = data['Solution__c'].astype(str)
# data['Definition__c'] = data['Definition__c'].astype(str)
# data['Answer__c'] = data['Answer__c'].astype(str)
# data['Problem__c'] = data['Problem__c'].astype(str)
# data['How_To__c'] = data['How_To__c'].astype(str)

In [34]:
# final_list = []
# for i in range(len(data)):
#     if(pd.isnull(data['Definition__c'][i])):
#         final_list.append('None')
    
#     else:
#         final_list.append(data['Definition__c'][i] + data['Solution__c'][i])

### Combining the Answer and Solution data fileds

In [5]:
final_list = []
for i in range(len(data)):
    if(pd.isnull(data['Answer__c'][i])):
        if(pd.isnull(data['Solution__c'][i])):
            final_list.append('None')
        else:
            final_list.append(data['Solution__c'][i])
    
    else:
        final_list.append(data['Answer__c'][i])


In [6]:
data["Solution"] = final_list

In [7]:
data

,Name,Answer__c,CreatedDate,Article_Number__c,Author_Region__c,Cadence_Platform__c,Channel__c,Critical__c,Definition__c,External_Family__c,...,Product_Feature2__r.Name,Product_Feature_Lvl_1__r.Name,Product_Feature_Lvl_2__r.Name,Product_Feature_Version__r.Name,Product_Feature__r.Name,Product__r.Name,Published_Date__c,Saved_Solution__c,Solution__c,Solution
0,20467935,NaN,2017-06-13T06:27:28.000+0000,538672.0,GCS,Functional Verification,Customer,0.0,<p><strong>For VHDL:</strong></p>\n\n<p> Yo...,Xcelium,...,Single Core,Multi-Language Sim,LRM Compliance,None,Single Core,Xcelium,2017-07-14T09:00:55.000+0000,<p>This is correct behavior as per the LRM as ...,<p>This is correct behavior as per the LRM as ...,<p>This is correct behavior as per the LRM as ...
1,20467953,NaN,2017-06-13T21:36:58.000+0000,538690.0,NACS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Single Core,UVM,UVM SV,None,Single Core,Xcelium,2017-06-13T22:11:03.000+0000,<p>It is not possible to directly pass an enum...,<p>It is not possible to directly pass an enum...,<p>It is not possible to directly pass an enum...
2,20467959,NaN,2017-06-14T13:39:23.000+0000,538696.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Multi Core,Documentation,NaN,20.03,Multi Core,Xcelium,2017-06-14T13:39:23.000+0000,NaN,NaN,None
3,20467982,NaN,2017-06-15T11:18:38.000+0000,538719.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Single Core,Comprehensive Coverage,Simulation/Recording,20.03,Single Core,Xcelium,2017-06-15T11:18:38.000+0000,NaN,NaN,None
4,20467983,NaN,2017-06-15T11:49:36.000+0000,538720.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Functional Safety,Documentation,NaN,17.04,Functional Safety,Xcelium,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,20464289,"<p>Most likely, the issue is in the user C cod...",2017-02-15T06:39:10.000+0000,535026.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Enterprise Simulator,SystemVerilog Sim,Simulator Engine,None,Single Core,Xcelium,2017-04-04T13:30:34.000+0000,NaN,NaN,"<p>Most likely, the issue is in the user C cod..."
3336,20464503,NaN,2017-02-21T05:27:25.000+0000,535240.0,NACS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Enterprise Simulator,Comprehensive Coverage,Functional,None,Single Core,Xcelium,2017-03-01T13:28:57.000+0000,<p>Assertions in simulation can be turned off ...,<p>Assertions in simulation can be turned off ...,<p>Assertions in simulation can be turned off ...
3337,20464541,NaN,2017-02-22T09:21:36.000+0000,535278.0,GCS,Functional Verification,Customer,0.0,"<p>The &quot;<span style=""font-family: courier...",Xcelium,...,Enterprise Simulator,UVM,UVM_REG,None,Single Core,Xcelium,2017-04-13T07:51:44.000+0000,<p>If you want to enable coverage for the envi...,<p>If you want to enable coverage for the envi...,<p>If you want to enable coverage for the envi...
3338,20464805,<p>There is an option called &quot;<span style...,2017-02-25T11:54:23.000+0000,535542.0,NACS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Enterprise Simulator,Multi-Language Sim,TCL,None,Single Core,Xcelium,2017-03-07T20:14:28.000+0000,NaN,NaN,<p>There is an option called &quot;<span style...


In [8]:
final_list = []
for i in range(len(data)):
    if(pd.isnull(data['Problem__c'][i])):
        if(pd.isnull(data['How_To__c'][i])):
            if(pd.isnull(data['Definition__c'][i])):
                final_list.append('None')
            else:
                final_list.append(data['Definition__c'][i])
        
        else:
            final_list.append(data['How_To__c'][i])
    
    else:
        final_list.append(data['Problem__c'][i])

data['Problem'] = final_list

In [9]:
data

,Name,Answer__c,CreatedDate,Article_Number__c,Author_Region__c,Cadence_Platform__c,Channel__c,Critical__c,Definition__c,External_Family__c,...,Product_Feature_Lvl_1__r.Name,Product_Feature_Lvl_2__r.Name,Product_Feature_Version__r.Name,Product_Feature__r.Name,Product__r.Name,Published_Date__c,Saved_Solution__c,Solution__c,Solution,Problem
0,20467935,NaN,2017-06-13T06:27:28.000+0000,538672.0,GCS,Functional Verification,Customer,0.0,<p><strong>For VHDL:</strong></p>\n\n<p> Yo...,Xcelium,...,Multi-Language Sim,LRM Compliance,None,Single Core,Xcelium,2017-07-14T09:00:55.000+0000,<p>This is correct behavior as per the LRM as ...,<p>This is correct behavior as per the LRM as ...,<p>This is correct behavior as per the LRM as ...,<p><strong>For VHDL:</strong></p>\n\n<p> Yo...
1,20467953,NaN,2017-06-13T21:36:58.000+0000,538690.0,NACS,Functional Verification,Customer,0.0,NaN,Xcelium,...,UVM,UVM SV,None,Single Core,Xcelium,2017-06-13T22:11:03.000+0000,<p>It is not possible to directly pass an enum...,<p>It is not possible to directly pass an enum...,<p>It is not possible to directly pass an enum...,<p>User is attempting to set an enumerated typ...
2,20467959,NaN,2017-06-14T13:39:23.000+0000,538696.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Documentation,NaN,20.03,Multi Core,Xcelium,2017-06-14T13:39:23.000+0000,NaN,NaN,None,None
3,20467982,NaN,2017-06-15T11:18:38.000+0000,538719.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Comprehensive Coverage,Simulation/Recording,20.03,Single Core,Xcelium,2017-06-15T11:18:38.000+0000,NaN,NaN,None,None
4,20467983,NaN,2017-06-15T11:49:36.000+0000,538720.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Documentation,NaN,17.04,Functional Safety,Xcelium,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,20464289,"<p>Most likely, the issue is in the user C cod...",2017-02-15T06:39:10.000+0000,535026.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,SystemVerilog Sim,Simulator Engine,None,Single Core,Xcelium,2017-04-04T13:30:34.000+0000,NaN,NaN,"<p>Most likely, the issue is in the user C cod...",<p>How can I debug intermittent regression cra...
3336,20464503,NaN,2017-02-21T05:27:25.000+0000,535240.0,NACS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Comprehensive Coverage,Functional,None,Single Core,Xcelium,2017-03-01T13:28:57.000+0000,<p>Assertions in simulation can be turned off ...,<p>Assertions in simulation can be turned off ...,<p>Assertions in simulation can be turned off ...,<p>The coverage database that was dumped from ...
3337,20464541,NaN,2017-02-22T09:21:36.000+0000,535278.0,GCS,Functional Verification,Customer,0.0,"<p>The &quot;<span style=""font-family: courier...",Xcelium,...,UVM,UVM_REG,None,Single Core,Xcelium,2017-04-13T07:51:44.000+0000,<p>If you want to enable coverage for the envi...,<p>If you want to enable coverage for the envi...,<p>If you want to enable coverage for the envi...,"<p>The &quot;<span style=""font-family: courier..."
3338,20464805,<p>There is an option called &quot;<span style...,2017-02-25T11:54:23.000+0000,535542.0,NACS,Functional Verification,Customer,0.0,NaN,Xcelium,...,Multi-Language Sim,TCL,None,Single Core,Xcelium,2017-03-07T20:14:28.000+0000,NaN,NaN,<p>There is an option called &quot;<span style...,<p>How can I dump a specific portion of covera...


In [10]:
data = data.drop(data.columns[[1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 20, 21]], axis=1)
data.head(1)

,Name,Article_Number__c,Product_Feature2__r.Name,Product_Feature_Lvl_1__r.Name,Solution,Problem
0,20467935,538672.0,Single Core,Multi-Language Sim,<p>This is correct behavior as per the LRM as ...,<p><strong>For VHDL:</strong></p>\n\n<p> Yo...


In [11]:
a = []
for i in data['Solution']:
    j = BeautifulSoup(i).get_text()
    a.append(j)
    
data['Solution']=a
data.head()

,Name,Article_Number__c,Product_Feature2__r.Name,Product_Feature_Lvl_1__r.Name,Solution,Problem
0,20467935,538672.0,Single Core,Multi-Language Sim,This is correct behavior as per the LRM as std...,<p><strong>For VHDL:</strong></p>\n\n<p> Yo...
1,20467953,538690.0,Single Core,UVM,It is not possible to directly pass an enum. T...,<p>User is attempting to set an enumerated typ...
2,20467959,538696.0,Multi Core,Documentation,None,None
3,20467982,538719.0,Single Core,Comprehensive Coverage,None,None
4,20467983,538720.0,Functional Safety,Documentation,None,None


In [12]:
a = []
for i in data['Problem']:
    j = BeautifulSoup(i).get_text()
    a.append(j)
    
data['Problem']=a
data.head()

,Name,Article_Number__c,Product_Feature2__r.Name,Product_Feature_Lvl_1__r.Name,Solution,Problem
0,20467935,538672.0,Single Core,Multi-Language Sim,This is correct behavior as per the LRM as std...,For VHDL:\n You get the CSGMSS error for da...
1,20467953,538690.0,Single Core,UVM,It is not possible to directly pass an enum. T...,User is attempting to set an enumerated type v...
2,20467959,538696.0,Multi Core,Documentation,None,None
3,20467982,538719.0,Single Core,Comprehensive Coverage,None,None
4,20467983,538720.0,Functional Safety,Documentation,None,None


In [13]:
data['Solution'] = data['Solution'].str.replace(r'\n', '')
data['Problem'] = data['Problem'].str.replace(r'\n', '')

C:\Users\swani\AppData\Local\Temp/ipykernel_2400/478330520.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Solution'] = data['Solution'].str.replace(r'\n', '')
C:\Users\swani\AppData\Local\Temp/ipykernel_2400/478330520.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Problem'] = data['Problem'].str.replace(r'\n', '')


In [14]:
data.head()

,Name,Article_Number__c,Product_Feature2__r.Name,Product_Feature_Lvl_1__r.Name,Solution,Problem
0,20467935,538672.0,Single Core,Multi-Language Sim,This is correct behavior as per the LRM as std...,For VHDL: You get the CSGMSS error for data...
1,20467953,538690.0,Single Core,UVM,It is not possible to directly pass an enum. T...,User is attempting to set an enumerated type v...
2,20467959,538696.0,Multi Core,Documentation,None,None
3,20467982,538719.0,Single Core,Comprehensive Coverage,None,None
4,20467983,538720.0,Functional Safety,Documentation,None,None


### Remove Stopwords

In [15]:
stop_words = stopwords.words('english')
data['Problem'] = data['Problem'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
data['Solution'] = data['Solution'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

### Remove Keywords flagged by IT Stopwords

In [17]:
df = pd.read_excel('ALl-keywords-2021.xlsx')
exceptions = df["Keywords Flagged by IT"].tolist()
data['Problem'] = data['Problem'].apply(lambda x: ' '.join([word for word in x.split() if word not in (exceptions)]))
data['Solution'] = data['Solution'].apply(lambda x: ' '.join([word for word in x.split() if word not in (exceptions)]))

In [18]:
exceptions

['simulation',
 'ref',
 'taken ownership',
 'understanding',
 'support database',
 'cadence',
 'intended recipient',
 'Reference',
 'attachments',
 'ArticleAttachmentPortal',
 'compilation',
 'xcelium',
 'elaboration',
 'open attachments',
 'design challenges',
 'simulations',
 'Venkatesh Lingaiah',
 'unexpected condition',
 'Sumit Sharma',
 'customer',
 'webex session',
 'follow-up query',
 'dissemination',
 'workaround',
 'command',
 'Cadence Support',
 'customer support',
 'Design Systems',
 'customers',
 'Product Feature',
 'webex',
 'Rajat Gupta',
 'legally privileged',
 'email originated',
 'Cadence Design Systems',
 'extended help',
 'Western Digital',
 'ArticleContent',
 'warnings',
 'Hiren Shah',
 'Ashutosh Gupta',
 'exit',
 'sender',
 'Xcelium Product Feature',
 'Atul Kumar',
 'Vivek Raheja',
 'articles',
 'unauthorized review',
 'use of the individual',
 'root cause',
 'Regards',
 'Services Agreement',
 'privacy laws',
 'availability',
 'Xcelium Product',
 'salesforce',
 'Pr

In [19]:
data.head()

,Name,Article_Number__c,Product_Feature2__r.Name,Product_Feature_Lvl_1__r.Name,Solution,Problem
0,20467935,538672.0,Single Core,Multi-Language Sim,"This correct behavior per LRM std_ulogic, std_...",For VHDL: You get CSGMSS error data types unre...
1,20467953,538690.0,Single Core,UVM,It possible directly pass enum. The main enum ...,User attempting set enumerated type variable t...
2,20467959,538696.0,Multi Core,Documentation,None,None
3,20467982,538719.0,Single Core,Comprehensive Coverage,None,None
4,20467983,538720.0,Functional Safety,Documentation,None,None


### Lemmatization

In [20]:
lmtzr = WordNetLemmatizer()
lemmatized = [[lmtzr.lemmatize(word) for word in word_tokenize(s)]
              for s in data['Problem']]
data['Problem'] = lemmatized
data['Problem'] = [' '.join(map(str, l)) for l in data['Problem']]

lemmatized = [[lmtzr.lemmatize(word) for word in word_tokenize(s)]
              for s in data['Solution']]
data['Solution'] = lemmatized
data['Solution'] = [' '.join(map(str, l)) for l in data['Solution']]

data.head()

,Name,Article_Number__c,Product_Feature2__r.Name,Product_Feature_Lvl_1__r.Name,Solution,Problem
0,20467935,538672.0,Single Core,Multi-Language Sim,"This correct behavior per LRM std_ulogic , std...",For VHDL : You get CSGMSS error data type unre...
1,20467953,538690.0,Single Core,UVM,It possible directly pas enum . The main enum ...,User attempting set enumerated type variable t...
2,20467959,538696.0,Multi Core,Documentation,None,None
3,20467982,538719.0,Single Core,Comprehensive Coverage,None,None
4,20467983,538720.0,Functional Safety,Documentation,None,None


In [21]:
datatoexcel = pd.ExcelWriter('N-Grams KB Articles Processes Data.xlsx')
  
# write DataFrame to excel
data.to_excel(datatoexcel)
  
# save the excel
datatoexcel.save()